In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas import DataFrame
import pickle
import os
from dtaidistance import dtw, dtw_visualisation as dtwvis
from dtaidistance import dtw_ndim

In [3]:
filenames_patients = {
    "cruz_ej1": [
        "cruz_ej1_0",
        "cruz_ej1_3",
        "cruz_ej1_13",
        "cruz_ej1_15",
        "cruz_ej1_35",
        "cruz_ej1_40",
        "cruz_ej1_55",
        "cruz_ej1_57",
        ],
    "cruz_ej2" : [
        "cruz_ej2_1",
        "cruz_ej2_14",
        "cruz_ej2_19",
        "cruz_ej2_23",
        "cruz_ej2_26",
        "cruz_ej2_28",
        "cruz_ej2_29",
        "cruz_ej2_36",
        "cruz_ej2_41",
        "cruz_ej2_45",
        "cruz_ej2_47",
        "cruz_ej2_49",
        "cruz_ej2_53",
        "cruz_ej2_60",
    ],
    "cruz_ej3" : [
        "cruz_ej3_2",
        "cruz_ej3_6",
        "cruz_ej3_10",
        "cruz_ej3_17",
        "cruz_ej3_59",
    ],
    "cruz_ej4" : [
        "cruz_ej4_30",
        "cruz_ej4_31",
        "cruz_ej4_32",
        "cruz_ej4_34",
        "cruz_ej4_38",
        "cruz_ej4_42",
        "cruz_ej4_51",
        "cruz_ej4_54",
    ],
    "pelota1" : [
        "pelota1_4",
        "pelota1_0",
        "pelota1_11",
        "pelota1_17",
        "pelota1_20",
        "pelota1_24",
        "pelota1_30",
        "pelota1_31",
    ],
    "pelota2": [
        "pelota2_12",
        "pelota2_32",
    ]
}    

filenames_correct = {
    "cruz_ej1" : [
        "josemi_cruz1"
    ],
    "cruz_ej2" : [
        "josemi_cruz2"
    ],
    "cruz_ej3" : [
        "josemi_cruz3",
        "joselu_cruz3"
    ],
    "cruz_ej4" : [
        "josemi_cruz4",
    ],
    "pelota1" : [
        "josemi_pelota1",
        "joselu_pelota1",
        "gadea_pelota1"
    ],
    "pelota2" : [
        "josemi_pelota2",
        "joselu_pelota2",
        "gadea_pelota2"
    ]
}


In [4]:
def get_filenames_patients(exercise: str, df = True):
    extension = ".df"
    if not df:
        extension = ".csv"
    
    sufix = ".pos"
    dataframes = []
    for i in filenames_patients[exercise]:
        actual_df =  i + sufix + extension
        dataframes.append(actual_df)
    return dataframes

def get_filenames_profesionals(exercise: str):
    sufix = ".df"
    dataframes = []
    for i in filenames_correct[exercise]:
        actual_df = i + sufix
        dataframes.append(actual_df)
    return dataframes

In [5]:
DATA_DIR = "../datos"
CSV_DIR = "CSV"
DATAFRAME_DIR = "DataFrames"
PROFESIOLS_DIR = "Correctos"
NORMALIZED = "normalized"
def open_dataframe(df_name):
    df: DataFrame
    df_name = os.path.join(NORMALIZED, DATAFRAME_DIR, df_name)
    
    with open(df_name, 'rb') as fp:
        df = pd.read_pickle(fp)
    df = df.dropna(subset=df.columns.values)
    df = df.reset_index(drop=True)
    return df

def open_numpy(numpy_name):
    file = None
    numpy_name = os.path.join(NORMALIZED, PROFESIOLS_DIR, numpy_name)

    with open(numpy_name, 'rb') as fp:
        file = pd.read_pickle(fp)
    file = file.dropna(subset=file.columns.values)
    file = file.reset_index(drop=True)
    return file

In [6]:
USEDPARTS = "usedparts"
def saved_used_parts(ej, parts):
    filename_df =  os.path.join(USEDPARTS) + ".df"
    filename_csv =  os.path.join(USEDPARTS) + ".json"
    data = pd.DataFrame({})   
    with open(filename_df, "ab") as file:
        pass
    with open(filename_df, "rb") as file:
        try:
            data = pickle.load(file)
        except:
            pass
    with open(filename_df, "wb") as file:
        data[ej] = {"coords": parts}
        pickle.dump(data, file, protocol=pickle.HIGHEST_PROTOCOL)
        data.to_json(filename_csv)

def open_used_parts(ej):
    filename_df =  os.path.join(USEDPARTS) + ".df"
    with open(filename_df, "rb") as file:
        try:
            data = pickle.load(file)
            return data[ej]
        except:
            pass

In [7]:
body = ['nariz', 'cuello', 'hombroI', 'hombroD',
       'codoI', 'codoD', 'manoI', 'manoD', 'caderaI', 'caderaD',
       'caderaM', 'rodillaI', 'rodillaD', 'tobilloI', 'tobilloD']


def get_positions(array):
    positions = []
    for i in array:
        positions.append(i + "_x")
        positions.append(i + "_y")
    return positions

def get_positions_x(array):
        positions = []
        for i in array:
            positions.append(i + "_x")
        return positions

def get_positions_y(array):
        positions = []
        for i in array:
            positions.append(i + "_y")
        return positions

In [8]:
def get_score(distance):
    score = 100 - 100*distance 
    return score if score>0 else 0 

In [12]:
def dtwdistance(user1, user2):
    array1 = np.asarray(user1).astype('float64').reshape(2*len(user1),)
    array2 = np.asarray(user2).astype('float64').reshape(2*len(user2),)
    distance = dtw.distance_fast(array1, array2)
    return distance

def dtwmult(user1, user2):
    from dtaidistance import dtw_ndim
    return dtw_ndim.distance_fast(np.array(user1).transpose(),np.array(user2).transpose())

def compare_exercises1(data, profesionals, patients, ej_name, all_coords_names, used_coords_names):
    for ej1, profesional in profesionals.items():
        for ej2, patient in patients.items():
            temp = []
            
            all_coords_distance = 0
            used_coords_distance = 0

            temp.append(ej_name)
            temp.append(ej1)
            temp.append(ej2)

            for coord_name in all_coords_names:
                coord_names = [coord_name+"_x", coord_name+"_y"]
                distance = dtwdistance(profesional[coord_names], patient[coord_names])
                #temp.append(distance)
                all_coords_distance += distance

                if coord_name in used_coords_names:
                    used_coords_distance += distance

            used_flatten_coords_pat = []
            used_flatten_coords_prof = []
            all_flatten_coords_pat = []
            all_flatten_coords_prof = []
            for coord_name in all_coords_names:
                coord_names = [coord_name+"_x", coord_name+"_y"]
                temp_pat_coords = patient[coord_names]
                temp_prof_coords = profesional[coord_names]
                
                temp_pat_coords = np.asarray(temp_pat_coords).astype('float64').reshape(2*len(temp_pat_coords),)
                temp_prof_coords = np.asarray(temp_prof_coords).astype('float64').reshape(2*len(temp_prof_coords),)
                
                all_flatten_coords_pat.append(temp_pat_coords)
                all_flatten_coords_prof.append(temp_prof_coords)

                if coord_name in used_coords_names:
                    used_flatten_coords_pat.append(temp_pat_coords)
                    used_flatten_coords_prof.append(temp_prof_coords)

            mult_all_distance = dtwmult(all_flatten_coords_prof, all_flatten_coords_pat)
            mult_used_distance = dtwmult(used_flatten_coords_prof, used_flatten_coords_pat)
            used_distance = used_coords_distance/len(used_coords_names)
            all_distance = all_coords_distance/len(all_coords_names)

            #temp.append(all_distance)
            #temp.append(used_distance)
            
            #temp.append(get_score(all_distance))
            temp.append(get_score(used_distance))
            

            #temp.append(get_score(mult_all_distance/len(all_coords_names)))
            temp.append(get_score(mult_used_distance/len(used_coords_names)))
    
            data.loc[len(data)] = temp

def comparisons1(data, ej_name):
    all_coords_names = body
    used_coords_names: pd.DataFrame = open_used_parts(ej_name)["coords"]

    dataframes_pelota2_patients = get_filenames_patients(ej_name)
    patients = {}
    for ej2 in dataframes_pelota2_patients:
        temp = open_dataframe(ej2)
        temp = temp[get_positions(body)]
        patients[ej2] = temp

    dataframes_pelota2_profesionals = get_filenames_profesionals(ej_name)
    profesionals = {}
    for ej2 in dataframes_pelota2_profesionals:
        temp = open_numpy(ej2)
        temp = temp[get_positions(body)]
        profesionals[ej2] = temp
    
    compare_exercises1(data, profesionals, profesionals, ej_name, all_coords_names, used_coords_names)
    compare_exercises1(data, profesionals, patients, ej_name, all_coords_names, used_coords_names)   
            
def save_all1():
    for ej in filenames_patients.keys():
        print(ej)

        data = pd.DataFrame()
        data["ej"] = []
        data["profesional"] = []
        data["patient"] = []
        #for coord in body:
        #    data[coord] = []
        #data["all_parts_distance"] = []
        #data["used_parts_distance"] = []
        #data["all_parts_score"] = []
        data["used_parts_score"] = []
        #data["all_multiplesdim_score"] = []
        data["used_multiplesdim_score"] = []

        comparisons1(data, ej)
        display(data)
        with open(os.path.join("results","coords",ej+".df"), "wb") as file:
            pickle.dump(data, file, protocol=pickle.HIGHEST_PROTOCOL)
            data.to_csv(os.path.join("results","coords",ej+".csv"))


save_all1()

cruz_ej1


,ej,profesional,patient,used_parts_score,used_multiplesdim_score
0,cruz_ej1,josemi_cruz1.df,josemi_cruz1.df,100.000000,100.000000
1,cruz_ej1,josemi_cruz1.df,cruz_ej1_0.pos.df,36.915808,69.999992
2,cruz_ej1,josemi_cruz1.df,cruz_ej1_3.pos.df,29.112615,66.125350
3,cruz_ej1,josemi_cruz1.df,cruz_ej1_13.pos.df,28.502646,66.469242
4,cruz_ej1,josemi_cruz1.df,cruz_ej1_15.pos.df,23.218323,63.798944
5,cruz_ej1,josemi_cruz1.df,cruz_ej1_35.pos.df,48.775422,77.228030
6,cruz_ej1,josemi_cruz1.df,cruz_ej1_40.pos.df,71.279809,85.487510
7,cruz_ej1,josemi_cruz1.df,cruz_ej1_55.pos.df,62.918622,83.138108
8,cruz_ej1,josemi_cruz1.df,cruz_ej1_57.pos.df,67.087132,84.440327


cruz_ej2


,ej,profesional,patient,used_parts_score,used_multiplesdim_score
0,cruz_ej2,josemi_cruz2.df,josemi_cruz2.df,100.000000,100.000000
1,cruz_ej2,josemi_cruz2.df,cruz_ej2_1.pos.df,14.286089,51.438690
2,cruz_ej2,josemi_cruz2.df,cruz_ej2_14.pos.df,32.027256,63.518157
3,cruz_ej2,josemi_cruz2.df,cruz_ej2_19.pos.df,18.116149,54.321651
4,cruz_ej2,josemi_cruz2.df,cruz_ej2_23.pos.df,35.673969,67.379403
5,cruz_ej2,josemi_cruz2.df,cruz_ej2_26.pos.df,30.356109,63.221931
6,cruz_ej2,josemi_cruz2.df,cruz_ej2_28.pos.df,33.259594,64.901813
7,cruz_ej2,josemi_cruz2.df,cruz_ej2_29.pos.df,44.325714,72.097575
8,cruz_ej2,josemi_cruz2.df,cruz_ej2_36.pos.df,29.900608,62.561619
9,cruz_ej2,josemi_cruz2.df,cruz_ej2_41.pos.df,66.113573,81.918638


cruz_ej3


...

,ej,profesional,patient,used_parts_score,used_multiplesdim_score
0,cruz_ej4,josemi_cruz4.df,josemi_cruz4.df,100.000000,100.000000
1,cruz_ej4,josemi_cruz4.df,cruz_ej4_30.pos.df,41.521402,74.486518
2,cruz_ej4,josemi_cruz4.df,cruz_ej4_31.pos.df,33.830706,72.300412
3,cruz_ej4,josemi_cruz4.df,cruz_ej4_32.pos.df,46.509442,75.530623
4,cruz_ej4,josemi_cruz4.df,cruz_ej4_34.pos.df,65.047173,80.159434
5,cruz_ej4,josemi_cruz4.df,cruz_ej4_38.pos.df,40.539019,74.539369
6,cruz_ej4,josemi_cruz4.df,cruz_ej4_42.pos.df,41.701359,74.686027
7,cruz_ej4,josemi_cruz4.df,cruz_ej4_51.pos.df,59.905100,78.984748
8,cruz_ej4,josemi_cruz4.df,cruz_ej4_54.pos.df,49.134166,76.661278


pelota1


,ej,profesional,patient,used_parts_score,used_multiplesdim_score
0,pelota1,josemi_pelota1.df,josemi_pelota1.df,100.000000,100.000000
1,pelota1,josemi_pelota1.df,joselu_pelota1.df,83.104134,90.499872
2,pelota1,josemi_pelota1.df,gadea_pelota1.df,67.650969,82.266340
3,pelota1,joselu_pelota1.df,josemi_pelota1.df,83.104134,90.499872
4,pelota1,joselu_pelota1.df,joselu_pelota1.df,100.000000,100.000000
5,pelota1,joselu_pelota1.df,gadea_pelota1.df,67.038150,81.654816
6,pelota1,gadea_pelota1.df,josemi_pelota1.df,67.650969,82.266340
7,pelota1,gadea_pelota1.df,joselu_pelota1.df,67.038150,81.654816
8,pelota1,gadea_pelota1.df,gadea_pelota1.df,100.000000,100.000000
9,pelota1,josemi_pelota1.df,pelota1_4.pos.df,42.145171,54.272718


pelota2


,ej,profesional,patient,used_parts_score,used_multiplesdim_score
0,pelota2,josemi_pelota2.df,josemi_pelota2.df,100.000000,100.000000
1,pelota2,josemi_pelota2.df,joselu_pelota2.df,81.840098,91.116933
2,pelota2,josemi_pelota2.df,gadea_pelota2.df,68.602992,82.456640
3,pelota2,joselu_pelota2.df,josemi_pelota2.df,81.840098,91.116933
4,pelota2,joselu_pelota2.df,joselu_pelota2.df,100.000000,100.000000
5,pelota2,joselu_pelota2.df,gadea_pelota2.df,73.348629,85.243696
6,pelota2,gadea_pelota2.df,josemi_pelota2.df,68.602992,82.456640
7,pelota2,gadea_pelota2.df,joselu_pelota2.df,73.348629,85.243696
8,pelota2,gadea_pelota2.df,gadea_pelota2.df,100.000000,100.000000
9,pelota2,josemi_pelota2.df,pelota2_12.pos.df,49.149997,61.175503


In [10]:
def dtwdistance_sep(user1, user2):
    array1 = np.asarray(user1).astype('float64')
    array2 = np.asarray(user2).astype('float64')
    distance = dtw.distance_fast(array1, array2)
    return distance

def dtwdistance(user1, user2):
    array1 = np.asarray(user1).astype('float64').reshape(2*len(user1),)
    array2 = np.asarray(user2).astype('float64').reshape(2*len(user2),)
    distance = dtw.distance_fast(array1, array2)
    return distance

def dtwmult(user1, user2):
    from dtaidistance import dtw_ndim
    return dtw_ndim.distance_fast(np.array(user1).transpose(),np.array(user2).transpose())

def compare_exercises1(data, profesionals, patients, ej_name, all_coords_names, used_coords_names):
    for ej1, profesional in profesionals.items():
        for ej2, patient in patients.items():
            temp = []
            
            used_coords_distance = 0

            temp.append(ej_name)
            temp.append(ej1)
            temp.append(ej2)

            for coord_name in used_coords_names:
                coord_names = [coord_name+"_x", coord_name+"_y"]
                distance = dtwdistance(profesional[coord_names], patient[coord_names])
                used_coords_distance += distance
            used_distance = used_coords_distance/len(used_coords_names)

            temp.append(get_score(used_distance))

            used_coords_distance = 0

            for coord_name in used_coords_names:
                coord_name_x, coord_name_y = [coord_name+"_x", coord_name+"_y"]
                distance = dtwdistance_sep(profesional[coord_name_x], patient[coord_name_x])
                used_coords_distance += distance

                distance = dtwdistance_sep(profesional[coord_name_y], patient[coord_name_y])
                used_coords_distance += distance
            
            used_distance = used_coords_distance/(len(used_coords_names) * 2)
            temp.append(get_score(used_distance))


            used_flatten_coords_pat = []
            used_flatten_coords_prof = []
            for coord_name in used_coords_names:
                coord_names = [coord_name+"_x", coord_name+"_y"]
                temp_pat_coords = patient[coord_names]
                temp_prof_coords = profesional[coord_names]
                
                temp_pat_coords = np.asarray(temp_pat_coords).astype('float64').reshape(2*len(temp_pat_coords),)
                temp_prof_coords = np.asarray(temp_prof_coords).astype('float64').reshape(2*len(temp_prof_coords),)

                used_flatten_coords_pat.append(temp_pat_coords)
                used_flatten_coords_prof.append(temp_prof_coords)

            #mult_used_distance = dtwmult(used_flatten_coords_prof, used_flatten_coords_pat)
            #temp.append(get_score(mult_used_distance/(len(used_coords_names))))

            used_flatten_coords_pat = []
            used_flatten_coords_prof = []
            for coord_name in used_coords_names:
                coord_name_x = coord_name+"_x"
                temp_pat_coords = patient[coord_name_x]
                temp_prof_coords = profesional[coord_name_x]
                
                temp_pat_coords = np.asarray(temp_pat_coords).astype('float64').reshape(len(temp_pat_coords),)
                temp_prof_coords = np.asarray(temp_prof_coords).astype('float64').reshape(len(temp_prof_coords),)

                used_flatten_coords_pat.append(temp_pat_coords)
                used_flatten_coords_prof.append(temp_prof_coords)

            mult_used_distance = dtwmult(used_flatten_coords_prof, used_flatten_coords_pat)
            #temp.append(get_score(mult_used_distance/(len(used_coords_names))))

            used_flatten_coords_pat = []
            used_flatten_coords_prof = []
            for coord_name in used_coords_names:
                coord_name_y = coord_name+"_y"
                temp_pat_coords = patient[coord_name_y]
                temp_prof_coords = profesional[coord_name_y]
                
                temp_pat_coords = np.asarray(temp_pat_coords).astype('float64').reshape(len(temp_pat_coords),)
                temp_prof_coords = np.asarray(temp_prof_coords).astype('float64').reshape(len(temp_prof_coords),)

                used_flatten_coords_pat.append(temp_pat_coords)
                used_flatten_coords_prof.append(temp_prof_coords)

            mult_used_distance += dtwmult(used_flatten_coords_prof, used_flatten_coords_pat)
            temp.append(get_score(mult_used_distance/(len(used_coords_names))))




            data.loc[len(data)] = temp

def comparisons1(data, ej_name):
    all_coords_names = body
    used_coords_names: pd.DataFrame = open_used_parts(ej_name)["coords"]

    dataframes_pelota2_patients = get_filenames_patients(ej_name)
    patients = {}
    for ej2 in dataframes_pelota2_patients:
        temp = open_dataframe(ej2)
        temp = temp[get_positions(body)]
        patients[ej2] = temp

    dataframes_pelota2_profesionals = get_filenames_profesionals(ej_name)
    profesionals = {}
    for ej2 in dataframes_pelota2_profesionals:
        temp = open_numpy(ej2)
        temp = temp[get_positions(body)]
        profesionals[ej2] = temp
    
    compare_exercises1(data, profesionals, profesionals, ej_name, all_coords_names, used_coords_names)
    compare_exercises1(data, profesionals, patients, ej_name, all_coords_names, used_coords_names)   
            
def save_all1():
    for ej in filenames_patients.keys():
        print(ej)

        data = pd.DataFrame()
        data["ej"] = []
        data["profesional"] = []
        data["patient"] = []
        data["used_parts_score"] = []
        data["used_parts_score_sep"] = []
        #data["used_multiplesdim_score"] = []
        data["used_multiplesdim_score_sep"] = []

        comparisons1(data, ej)
        display(data)

        with open(os.path.join("results","coords",ej+".df"), "wb") as file:
            pickle.dump(data, file, protocol=pickle.HIGHEST_PROTOCOL)
            data.to_csv(os.path.join("results","coords",ej+".csv"))


save_all1()

cruz_ej1


,ej,profesional,patient,used_parts_score,used_parts_score_sep,used_multiplesdim_score_sep
0,cruz_ej1,josemi_cruz1.df,josemi_cruz1.df,100.000000,100.000000,100.000000
1,cruz_ej1,josemi_cruz1.df,cruz_ej1_0.pos.df,36.915808,37.776755,46.972301
2,cruz_ej1,josemi_cruz1.df,cruz_ej1_3.pos.df,29.112615,29.530332,40.948969
3,cruz_ej1,josemi_cruz1.df,cruz_ej1_13.pos.df,28.502646,26.183443,38.868493
4,cruz_ej1,josemi_cruz1.df,cruz_ej1_15.pos.df,23.218323,21.164913,34.712858
5,cruz_ej1,josemi_cruz1.df,cruz_ej1_35.pos.df,48.775422,41.727495,51.264520
6,cruz_ej1,josemi_cruz1.df,cruz_ej1_40.pos.df,71.279809,43.112392,54.453713
7,cruz_ej1,josemi_cruz1.df,cruz_ej1_55.pos.df,62.918622,42.707438,53.544811
8,cruz_ej1,josemi_cruz1.df,cruz_ej1_57.pos.df,67.087132,44.053527,54.758104


cruz_ej2


,ej,profesional,patient,used_parts_score,used_parts_score_sep,used_multiplesdim_score_sep
0,cruz_ej2,josemi_cruz2.df,josemi_cruz2.df,100.000000,100.000000,100.000000
1,cruz_ej2,josemi_cruz2.df,cruz_ej2_1.pos.df,14.286089,19.816868,21.474131
2,cruz_ej2,josemi_cruz2.df,cruz_ej2_14.pos.df,32.027256,37.008221,41.420455
3,cruz_ej2,josemi_cruz2.df,cruz_ej2_19.pos.df,18.116149,24.499025,27.092535
4,cruz_ej2,josemi_cruz2.df,cruz_ej2_23.pos.df,35.673969,43.249696,47.765706
5,cruz_ej2,josemi_cruz2.df,cruz_ej2_26.pos.df,30.356109,38.637979,41.764236
6,cruz_ej2,josemi_cruz2.df,cruz_ej2_28.pos.df,33.259594,41.671550,44.299131
7,cruz_ej2,josemi_cruz2.df,cruz_ej2_29.pos.df,44.325714,50.946128,54.993610
8,cruz_ej2,josemi_cruz2.df,cruz_ej2_36.pos.df,29.900608,37.249954,40.143942
9,cruz_ej2,josemi_cruz2.df,cruz_ej2_41.pos.df,66.113573,57.087063,62.165113


cruz_ej3


...

,ej,profesional,patient,used_parts_score,used_parts_score_sep,used_multiplesdim_score_sep
0,cruz_ej4,josemi_cruz4.df,josemi_cruz4.df,100.000000,100.000000,100.000000
1,cruz_ej4,josemi_cruz4.df,cruz_ej4_30.pos.df,41.521402,46.033558,54.313354
2,cruz_ej4,josemi_cruz4.df,cruz_ej4_31.pos.df,33.830706,39.246910,48.978836
3,cruz_ej4,josemi_cruz4.df,cruz_ej4_32.pos.df,46.509442,48.088680,56.312103
4,cruz_ej4,josemi_cruz4.df,cruz_ej4_34.pos.df,65.047173,55.133866,62.335011
5,cruz_ej4,josemi_cruz4.df,cruz_ej4_38.pos.df,40.539019,44.389593,53.457874
6,cruz_ej4,josemi_cruz4.df,cruz_ej4_42.pos.df,41.701359,45.790213,53.956511
7,cruz_ej4,josemi_cruz4.df,cruz_ej4_51.pos.df,59.905100,53.128659,61.008791
8,cruz_ej4,josemi_cruz4.df,cruz_ej4_54.pos.df,49.134166,50.262434,58.015028


pelota1


,ej,profesional,patient,used_parts_score,used_parts_score_sep,used_multiplesdim_score_sep
0,pelota1,josemi_pelota1.df,josemi_pelota1.df,100.000000,100.000000,100.000000
1,pelota1,josemi_pelota1.df,joselu_pelota1.df,83.104134,96.284060,96.113509
2,pelota1,josemi_pelota1.df,gadea_pelota1.df,67.650969,91.121989,91.631294
3,pelota1,joselu_pelota1.df,josemi_pelota1.df,83.104134,96.284060,96.113509
4,pelota1,joselu_pelota1.df,joselu_pelota1.df,100.000000,100.000000,100.000000
5,pelota1,joselu_pelota1.df,gadea_pelota1.df,67.038150,91.846934,91.277787
6,pelota1,gadea_pelota1.df,josemi_pelota1.df,67.650969,91.121989,91.631294
7,pelota1,gadea_pelota1.df,joselu_pelota1.df,67.038150,91.846934,91.277787
8,pelota1,gadea_pelota1.df,gadea_pelota1.df,100.000000,100.000000,100.000000
9,pelota1,josemi_pelota1.df,pelota1_4.pos.df,42.145171,49.332787,40.704072


pelota2


,ej,profesional,patient,used_parts_score,used_parts_score_sep,used_multiplesdim_score_sep
0,pelota2,josemi_pelota2.df,josemi_pelota2.df,100.000000,100.000000,100.000000
1,pelota2,josemi_pelota2.df,joselu_pelota2.df,81.840098,92.725514,93.668106
2,pelota2,josemi_pelota2.df,gadea_pelota2.df,68.602992,88.956915,89.091295
3,pelota2,joselu_pelota2.df,josemi_pelota2.df,81.840098,92.725514,93.668106
4,pelota2,joselu_pelota2.df,joselu_pelota2.df,100.000000,100.000000,100.000000
5,pelota2,joselu_pelota2.df,gadea_pelota2.df,73.348629,93.968512,93.170617
6,pelota2,gadea_pelota2.df,josemi_pelota2.df,68.602992,88.956915,89.091295
7,pelota2,gadea_pelota2.df,joselu_pelota2.df,73.348629,93.968512,93.170617
8,pelota2,gadea_pelota2.df,gadea_pelota2.df,100.000000,100.000000,100.000000
9,pelota2,josemi_pelota2.df,pelota2_12.pos.df,49.149997,59.008579,49.838080
